In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

from sklearn import metrics
from sklearn import svm
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score

from scipy import stats

from datetime import datetime

from mlxtend.regressor import StackingCVRegressor

In [124]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer

In [125]:
train = pd.read_csv("./data/train.csv")
train_y = train[["SALE PRICE"]]
train_X = train

test_X = pd.read_csv("./data/test.csv")
test_y = pd.read_csv("./data/test_groundtruth.csv")

print("train_X:",train_X.shape)
print("train_y:",train_y.shape)
print("test_X:",test_X.shape)
print("test_y:",test_y.shape)

train_X: (43064, 20)
train_y: (43064, 1)
test_X: (10767, 19)
test_y: (10767, 1)


In [126]:
train.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,R4,402 EAST 90TH STREET,5A,10128,1,0,1,-,-,1910,2,R4,1200000,2016-09-30 00:00:00
1,4,HOLLISWOOD,10 COOPS - ELEVATOR APARTMENTS,2,10538,70,D4,"87-50 204TH STREET, B42",,11423,0,0,0,-,-,1954,2,D4,120000,2017-02-10 00:00:00
2,3,DOWNTOWN-FULTON MALL,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,R4,265 STATE STREET,910,11201,1,0,1,0,0,2014,2,R4,1578287,2017-01-25 00:00:00
3,4,FLUSHING-NORTH,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,R4,137-11 32 AVENUE,4W,11354,1,0,1,-,-,0,2,R4,510000,2017-03-17 00:00:00
4,3,PARK SLOPE,09 COOPS - WALKUP APARTMENTS,2C,1067,29,C6,"862 PRESIDENT STREET, 1",,11215,0,0,0,0,0,1920,2,C6,1950000,2016-09-09 00:00:00


In [127]:
num_train_samples = len(train_X)

data_X = pd.concat([train_X, test_X])

# Data Pre Processing

In [128]:
# Let's delete some of the columns that we ** may not ** need
# 请注意 下面删除的特征很可能是有用的，合理的处理能够获得更为准确的预测模型，请探索所删除特征的使用
del data_X['ADDRESS']
del data_X['APARTMENT NUMBER']
del data_X['SALE DATE']
"""del data_X['BUILDING CLASS AT PRESENT']
del data_X['BUILDING CLASS AT TIME OF SALE']
del data_X['NEIGHBORHOOD']
del data_X['LAND SQUARE FEET']
del data_X['GROSS SQUARE FEET'] """

#data_X['SALE DATE'] = pd.to_datetime(data_X['SALE DATE']).astype(np.int64)

"del data_X['BUILDING CLASS AT PRESENT']\ndel data_X['BUILDING CLASS AT TIME OF SALE']\ndel data_X['NEIGHBORHOOD']\ndel data_X['LAND SQUARE FEET']\ndel data_X['GROSS SQUARE FEET'] "

In [129]:
# 将无法转换为数字的字符串转换为NaN
def convert_to_number(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

data_X['LAND SQUARE FEET'] = data_X['LAND SQUARE FEET'].map(convert_to_number)
data_X['GROSS SQUARE FEET'] = data_X['GROSS SQUARE FEET'].map(convert_to_number)
data_X['YEAR BUILT'] = data_X['YEAR BUILT'].map(convert_to_number)

data_X['LAND SQUARE FEET'].replace(0, np.nan, inplace=True)
data_X['GROSS SQUARE FEET'].replace(0, np.nan, inplace=True)
data_X['YEAR BUILT'].replace(0, np.nan, inplace=True)

# 按NEIGHBORHOOD的中位数填充，如果某个NEIGHBORHOOD的所有LAND SQUARE FEET、GROSS SQUARE FEET或YEAR BUILT数据都是缺失的，则使用全局中位数填充
data_X['LAND SQUARE FEET'] = data_X.groupby('NEIGHBORHOOD')['LAND SQUARE FEET'].transform(
    lambda x: x.fillna(x[x>0].median()) if x[x>0].median() >= 0 
                                        else data_X['LAND SQUARE FEET'][data_X['LAND SQUARE FEET']>0].median())
data_X['GROSS SQUARE FEET'] = data_X.groupby('NEIGHBORHOOD')['GROSS SQUARE FEET'].transform(
    lambda x: x.fillna(x[x>0].median()) if x[x>0].median() >= 0 
                                        else data_X['GROSS SQUARE FEET'][data_X['GROSS SQUARE FEET']>0].median())
data_X['YEAR BUILT'] = data_X.groupby('NEIGHBORHOOD')['YEAR BUILT'].transform(
    lambda x: x.fillna(x[x>0].median()) if x[x>0].median() >= 0 
                                        else data_X['YEAR BUILT'][data_X['YEAR BUILT']>0].median())
    

# 数据类型转换
data_X['LAND SQUARE FEET'] = data_X['LAND SQUARE FEET'].astype(np.int64)
data_X['GROSS SQUARE FEET'] = data_X['GROSS SQUARE FEET'].astype(np.int64)
data_X['YEAR BUILT'] = data_X['YEAR BUILT'].astype(np.int64)
#data_X['HOUSE AGE'] = 2017 - data_X['YEAR BUILT']
#data_X['HAS_APARTMENT_NUMBER'] = ~data_X['APARTMENT NUMBER'].isnull()


In [130]:
Neighbor_Group = data_X.groupby(['NEIGHBORHOOD'])[['SALE PRICE']].agg(['mean'])
Neighbor_Dict = dict(dict(Neighbor_Group[('SALE PRICE', 'mean')]))
Borough_Group = data_X.groupby(['BOROUGH'])[['SALE PRICE']].agg(['mean'])
Borough_Dict = dict(Borough_Group[('SALE PRICE', 'mean')])
BCP_Group = data_X.groupby(['BUILDING CLASS AT PRESENT'])[['SALE PRICE']].agg(['mean'])
BCP_Dict = dict(BCP_Group[('SALE PRICE', 'mean')])
BCA_Group = data_X.groupby(['BUILDING CLASS AT TIME OF SALE'])[['SALE PRICE']].agg(['mean'])
BCA_Dict = dict(BCA_Group[('SALE PRICE', 'mean')])
TCP_Group = data_X.groupby(['TAX CLASS AT PRESENT'])[['SALE PRICE']].agg(['mean'])
TCP_Dict = dict(TCP_Group[('SALE PRICE', 'mean')])
TCA_Group = data_X.groupby(['TAX CLASS AT TIME OF SALE'])[['SALE PRICE']].agg(['mean'])
TCA_Dict = dict(TCA_Group[('SALE PRICE', 'mean')])

data_X["oNeighborhood"] = data_X["NEIGHBORHOOD"].map(Neighbor_Dict).fillna(1).apply(np.log)
data_X["oBOR"] = data_X["BOROUGH"].map(Borough_Dict).fillna(1).apply(np.log)
data_X["oBCA"] = data_X["BUILDING CLASS AT TIME OF SALE"].map(BCA_Dict).fillna(1).apply(np.log)
data_X["oBCP"] = data_X["BUILDING CLASS AT PRESENT"].map(BCP_Dict).fillna(1).apply(np.log)
data_X["oTCA"] = data_X["BUILDING CLASS AT TIME OF SALE"].map(TCA_Dict).fillna(1).apply(np.log)
data_X["oTCP"] = data_X["BUILDING CLASS AT PRESENT"].map(TCP_Dict).fillna(1).apply(np.log)

""" data_X.drop(['BOROUGH'], axis=1, inplace=True)
data_X.drop(['BUILDING CLASS AT TIME OF SALE'], axis=1, inplace=True)
data_X.drop(['BUILDING CLASS AT PRESENT'], axis=1, inplace=True)
data_X.drop(['NEIGHBORHOOD'], axis=1, inplace=True) """

" data_X.drop(['BOROUGH'], axis=1, inplace=True)\ndata_X.drop(['BUILDING CLASS AT TIME OF SALE'], axis=1, inplace=True)\ndata_X.drop(['BUILDING CLASS AT PRESENT'], axis=1, inplace=True)\ndata_X.drop(['NEIGHBORHOOD'], axis=1, inplace=True) "

In [131]:
NumStr = [
    "YEAR BUILT",
    "TAX CLASS AT PRESENT",
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS CATEGORY",
    ]
for col in NumStr:
    data_X[col] = data_X[col].astype(str)

data_X.drop(['SALE PRICE'], axis=1, inplace=True)

In [132]:
# Let's convert some of the columns to appropriate datatype

data_X['TAX CLASS AT TIME OF SALE'] = data_X['TAX CLASS AT TIME OF SALE'].astype('category')
data_X['TAX CLASS AT PRESENT'] = data_X['TAX CLASS AT PRESENT'].astype('category')
data_X['BUILDING CLASS CATEGORY'] = data_X['BUILDING CLASS CATEGORY'].astype('category')
data_X['BOROUGH'] = data_X['BOROUGH'].astype('category')
#data_X['ADDRESS'] = data_X['ADDRESS'].astype('category')
#data_X['APARTMENT NUMBER'] = data_X['APARTMENT NUMBER'].astype('category')
data_X['BUILDING CLASS AT PRESENT'] = data_X['BUILDING CLASS AT PRESENT'].astype('category')
data_X['BUILDING CLASS AT TIME OF SALE'] = data_X['BUILDING CLASS AT TIME OF SALE'].astype('category')
data_X['NEIGHBORHOOD'] = data_X['NEIGHBORHOOD'].astype('category')

In [133]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53831 entries, 0 to 10766
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   BOROUGH                         53831 non-null  category
 1   NEIGHBORHOOD                    53831 non-null  category
 2   BUILDING CLASS CATEGORY         53831 non-null  category
 3   TAX CLASS AT PRESENT            53831 non-null  category
 4   BLOCK                           53831 non-null  int64   
 5   LOT                             53831 non-null  int64   
 6   BUILDING CLASS AT PRESENT       53831 non-null  category
 7   ZIP CODE                        53831 non-null  int64   
 8   RESIDENTIAL UNITS               53831 non-null  int64   
 9   COMMERCIAL UNITS                53831 non-null  int64   
 10  TOTAL UNITS                     53831 non-null  int64   
 11  LAND SQUARE FEET                53831 non-null  int64   
 12  GROSS SQUARE FEET 

In [134]:
data_X.head(10)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,...,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,oNeighborhood,oBOR,oBCA,oBCP,oTCA,oTCP
0,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,R4,10128,1,0,...,3141,1910,2,R4,14.116544,14.133208,14.114872,14.114216,0.0,0.0
1,4,HOLLISWOOD,10 COOPS - ELEVATOR APARTMENTS,2,10538,70,D4,11423,0,0,...,2055,1954,2,D4,13.145360,13.306605,13.411612,13.411205,0.0,0.0
2,3,DOWNTOWN-FULTON MALL,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,R4,11201,1,0,...,3300,2014,2,R4,14.299426,13.784905,14.114872,14.114216,0.0,0.0
3,4,FLUSHING-NORTH,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,R4,11354,1,0,...,1654,1962,2,R4,13.624013,13.306605,14.114872,14.114216,0.0,0.0
4,3,PARK SLOPE,09 COOPS - WALKUP APARTMENTS,2C,1067,29,C6,11215,0,0,...,3078,1920,2,C6,14.242159,13.784905,13.071034,13.071034,0.0,0.0
5,3,CLINTON HILL,12 CONDOS - WALKUP APARTMENTS,2,2014,1003,R2,11238,1,0,...,3200,1946,2,R2,13.912426,13.784905,13.378317,13.378317,0.0,0.0
6,4,BRIARWOOD,10 COOPS - ELEVATOR APARTMENTS,2,9648,24,D4,11435,0,0,...,1787,1954,2,D4,12.761068,13.306605,13.411612,13.411205,0.0,0.0
7,5,MIDLAND BEACH,05 TAX CLASS 1 VACANT LAND,1B,3816,1,V0,10306,0,0,...,1393,1981,1,V0,12.971578,13.084750,13.343691,13.311810,0.0,0.0
8,4,OZONE PARK,01 ONE FAMILY DWELLINGS,1,11398,4,A1,11417,1,0,...,1712,1920,1,A1,13.170211,13.306605,13.350937,13.351262,0.0,0.0
9,1,SOUTHBRIDGE,13 CONDOS - ELEVATOR APARTMENTS,2,91,1115,R4,10038,1,0,...,1800,1919,2,R4,14.065793,14.133208,14.114872,14.114216,0.0,0.0


In [135]:
class labelenc(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        lab=LabelEncoder()
        X["RESIDENTIAL UNITS"] = lab.fit_transform(X["RESIDENTIAL UNITS"])
        X["COMMERCIAL UNITS"] = lab.fit_transform(X["COMMERCIAL UNITS"])
        X["TOTAL UNITS"] = lab.fit_transform(X["GarageYrBlt"])
        return X

In [136]:
class skew_dummies(BaseEstimator, TransformerMixin):
    def __init__(self,skew=0.5):
        self.skew = skew
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X_numeric=X.select_dtypes(exclude=["object"])
        skewness = X_numeric.apply(lambda x: skew(x)) 
        skewness_features = skewness[abs(skewness) >= self.skew].index
        X[skewness_features] = np.log1p(X[skewness_features])
        X = pd.get_dummies(X)
        return X

In [137]:
pipe = Pipeline([
    ('labenc', labelenc()),
    ('skew_dummies', skew_dummies(skew=1)),
    ])

In [138]:
data_X.shape

(53831, 22)

In [139]:
#Select the variables to be one-hot encoded
one_hot_features = ['TAX CLASS AT PRESENT','TAX CLASS AT TIME OF SALE','BUILDING CLASS CATEGORY',
                    'BOROUGH','BUILDING CLASS AT PRESENT','BUILDING CLASS AT TIME OF SALE','NEIGHBORHOOD']

In [140]:
# Convert categorical variables into dummy/indicator variables (i.e. one-hot encoding).
one_hot_encoded = pd.get_dummies(data_X[one_hot_features])
one_hot_encoded.info(verbose=True, memory_usage=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53831 entries, 0 to 10766
Data columns (total 539 columns):
 #    Column                                                                Non-Null Count  Dtype
---   ------                                                                --------------  -----
 0    TAX CLASS AT PRESENT_                                                 53831 non-null  uint8
 1    TAX CLASS AT PRESENT_1                                                53831 non-null  uint8
 2    TAX CLASS AT PRESENT_1A                                               53831 non-null  uint8
 3    TAX CLASS AT PRESENT_1B                                               53831 non-null  uint8
 4    TAX CLASS AT PRESENT_1C                                               53831 non-null  uint8
 5    TAX CLASS AT PRESENT_2                                                53831 non-null  uint8
 6    TAX CLASS AT PRESENT_2A                                               53831 non-null  uint8
 7    TA

In [141]:
data_X = data_X.drop(one_hot_features,axis=1)
data_X = pd.concat([data_X, one_hot_encoded] ,axis=1)

In [142]:
train_X = data_X[:num_train_samples].to_numpy()
test_X = data_X[num_train_samples:].to_numpy()

# Regression

In [121]:
model1 = RandomForestRegressor()
"""model2 = xgb.XGBRegressor()
 voting_regr = VotingRegressor(estimators=[('rf', model1), ('lr', model2), ('ridge', model3)])
voting_regr.fit(train_X, train_y.values.ravel())
Y_pred_voting = voting_regr.predict(test_X) """

""" model1.fit(train_X, train_y.values.ravel())
Y_pred_1 = model1.predict(test_X)"""
model1.fit(train_X, train_y.values.ravel())
Y_pred_1 = model1.predict(test_X)

Y_pred_voting = Y_pred_1

mean_absolute_percentage_error(test_y,Y_pred_voting)

0.3470203003796817

In [116]:
pd.DataFrame({"pred":Y_pred_voting}).to_csv("2200017416_康子熙.csv")